In [11]:
import win32con
import pyautogui
import pyperclip
import win32gui
import time
import numpy as np
import cv2
import os
import sys

from datetime import datetime
from PIL import ImageGrab
from subprocess import Popen, PIPE
import glob
import pandas as pd
import signal
import win32process
import json
import subprocess
import re
from pykrx import stock


def press_keyboard(word):
    for p in word:
        pyautogui.press(p)
        time.sleep(0.1)

def imglocation(path, confidence = 0.7):
    n = np.fromfile(path, np.uint8)
    img = cv2.imdecode(n, cv2.IMREAD_COLOR)
    
    
    return pyautogui.locateOnScreen(img, confidence = confidence)

def create_error():
    a = 1 / 0

def get_processes_running(pid):
    # 영어일때는 `ignore` 인자 없어도 됨  
    tasks = subprocess.check_output(['tasklist']).decode('cp949', 'ignore').split("\r\n")
    p = []
    for task in tasks:
        m = re.match("(.+?) +(\d+) (.+?) +(\d+) +(\d+.* K).*",task)
        if m is not None:
            if m.group(2) == str(pid):
                return m.group(1)
    return None


class Kiwoom:
    def __init__(self):
        self.hwnd = 0
        self.open_list = []
        self.pid_list = []
        self.hts_name = None
        self.name = None
        self.hts_activate_flag = True
        os.chdir(r"C:\Users\kiwoom\jupyter")
        with open('hts_info.json', 'r') as f : 
            self.hts_info = json.load(f)

    def winEnumHandler(self, hwnd, ctx ):
        if win32gui.IsWindowVisible( hwnd ):
            self.open_list.append(win32gui.GetWindowText( hwnd ))
            
            tid, pid = win32process.GetWindowThreadProcessId(hwnd)
            self.pid_list.append(pid)

    def check_active_hts(self):
        win32gui.EnumWindows(self.winEnumHandler, None)

        for r in self.open_list:
            if self.hts_name in r:
                self.hwnd = 1
                break

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\KiwoomHero4\Bin\NKStarter.exe'], stdin=PIPE)
        time.sleep(5)
    
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\KiwoomHero4\조회전용 확인.png")
                if img_capture!= None:    
                    break
            except:
                pass
            time.sleep(1)
        
        press_keyboard(self.hts_info[self.name]['id'])
        time.sleep(3)
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\KiwoomHero4\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            time.sleep(1)
        time.sleep(10)

    def activate_hts(self):
        for win in pyautogui.getAllWindows():
            if self.hts_name in win.title:
                try:
                    win.activate()
                except:
                    win.minimize()
                    win.maximize()

    def download_file(self):
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=54, y=58)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('0191')
        time.sleep(5)
        pyautogui.press('enter')
        time.sleep(5)
        
        for _x, _rate in [(154, 20), (210, 30), (261, 40), (308, 50), (362, 60), (420, 100)]:
            ## 전체 클릭
            #pyautogui.moveTo(x=155, y=319)
            #pyautogui.click()
            #time.sleep(1)
            
            ## 20% 클릭
            pyautogui.moveTo(x=_x, y=317)
            pyautogui.click()
            time.sleep(1)
            
            ## 종목 중간에 커서 위치
            pyautogui.moveTo(x=391, y=554)
            time.sleep(1)
            
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            count = 0
            pre_img = None
            while True:
                img = ImageGrab.grab((145, 621,542, 793))
                if pre_img != None:
                    if pre_img == img:
                        count += 1
                        ## 열번 중복되면 break
                        if count == 20:
                            break
                    else:
                        count = 0
            
                pyautogui.scroll(-500)
                time.sleep(0.2)
                pre_img = img
                
            ## 오른쪽 클릭
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 엑셀로 저장 선택
            pyautogui.moveTo(x=527, y=983)
            pyautogui.click()
            time.sleep(1)
            
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'증거금{_rate}종목키움_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break        

    def close_hts(self):
        ## hts 닫기
        self.open_list = []
        self.pid_list = []
        win32gui.EnumWindows(self.winEnumHandler, None )
        for p, r in zip(self.pid_list, self.open_list):
            if self.hts_name in r:
                # pid로 종료
                try:
                    os.system("taskkill /pid "+str(p))
                    time.sleep(2.5)
                    pyautogui.press('enter')
                except:
                    pass
                    
                ## process 명으로 종료    
                try:
                    os.system("taskkill /im " + get_processes_running(p))
                    time.sleep(2.5)
                    pyautogui.press('enter')
                except:
                    pass
                break 

    def name_definition(self):
        self.name = '키움'
        self.hts_name = "영웅문"
        
    def run(self):
        time.sleep(2)
        self.name_definition()
        
        print(f"{self.name} 시작!!")
        
        self.check_active_hts()

        # ## hts가 켜져있지 않으면 login 과정 진행
        # if self.hwnd == 0:
        #     self.login()

        self.login()
        
        ## hts 창 활성화
        if self.hts_activate_flag:
            self.activate_hts()
            
        self.download_file()
        self.close_hts()


class KB(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\KB증권\hable\hable.exe'], stdin=PIPE)
        time.sleep(5)
    
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\KB증권\hable\조회전용 확인.png")
                if img_capture!= None:    
                    break
            except:
                pass
            time.sleep(1)
        
        press_keyboard(self.hts_info[self.name]['id'])
        pyautogui.press('tab')
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
    
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\KB증권\hable\조회전용안내.png")
                if img_capture!= None:    
                    break
            except:
                pass
            time.sleep(1)
    
        pyautogui.press('enter')
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\KB증권\hable\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            time.sleep(1)
        time.sleep(10)


    def download_file(self):
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=78, y=64)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('0191')
        time.sleep(5)
        pyautogui.press('enter')
        time.sleep(5)
        
        for _y, _rate in [(315, '증거금률차등'), (288, '증거금률100%')]:
            ## 업무구분 선택
            pyautogui.moveTo(x=323, y=269)
            pyautogui.click()
            time.sleep(1)
        
            ## 증거금차등종목 선택
            pyautogui.moveTo(x=314, y=_y,)
            pyautogui.click()
            time.sleep(1)

            #전체받기
            pyautogui.moveTo(x=543, y=267)
            pyautogui.click()
            time.sleep(5)
            
            
            ## 종목 사이 마우스 위치
            pyautogui.moveTo(x=452, y=506)
            time.sleep(10)
            
            
            ## 오른쪽 클릭
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 파일로 보내기 선택
            pyautogui.moveTo(x=589, y=755)
            pyautogui.click()
            time.sleep(1)
            
            ## csv 선택
            pyautogui.moveTo(x=788, y=838)
            pyautogui.click()
            time.sleep(1)
            
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'{_rate}종목kb_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
        
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=78, y=64)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('0942')
        time.sleep(5)
        pyautogui.press('enter')
        time.sleep(5)
        
        for _y, _rate in [(286, '신용가능'), (354, '주식대출')]:
            ## 구분 선택
            pyautogui.moveTo(x=352, y=264)
            pyautogui.click()
            time.sleep(1)
            
            ## 신용융자 선택
            pyautogui.moveTo(x=247, y=_y)
            pyautogui.click()
            time.sleep(1)
            
            ## 종목에 커서 위치
            pyautogui.moveTo(x=604, y=446)
            time.sleep(1)
            
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            count = 0
            pre_img = None
            while True:
                img = ImageGrab.grab((111, 526,1029, 656))
                if pre_img != None:
                    if pre_img == img:
                        count += 1
                        ## 열번 중복되면 break
                        if count == 20:
                            break
                    else:
                        count = 0
            
                pyautogui.scroll(-500)
                time.sleep(0.2)
                pre_img = img
            
            
            ## 오른쪽 클릭
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 파일로 보내기 선택
            pyautogui.moveTo(x=719, y=705)
            pyautogui.click()
            time.sleep(1)
            
            ## csv 선택
            pyautogui.moveTo(x=978, y=778)
            pyautogui.click()
            time.sleep(1)
            
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'{_rate}종목kb_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)    
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
                
    def name_definition(self):
        self.name = 'KB'
        self.hts_name = "KB증권"
        
class Shinhan(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\Shinhan\AlphaHTS\alphastarter.exe'], stdin=PIPE)
        time.sleep(5)
    
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\Shinhan\AlphaHTS\조회전용 확인.png")
                if img_capture!= None:    
                    break
            except:
                pass
            time.sleep(1)
        
        press_keyboard(self.hts_info[self.name]['id'])
        pyautogui.press('tab')
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\Shinhan\AlphaHTS\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            time.sleep(1)
        time.sleep(10)
        
    def download_file(self):
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=259, y=999)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('1806')
        time.sleep(5)
        pyautogui.press('enter')
        time.sleep(5)


        ## 현금 클릭
        pyautogui.moveTo(x=28, y=132)
        pyautogui.click()
        time.sleep(10)
        
        for _x, _rate in [(163, "30"), (218, "40"), (275, "50"), (329, "100_1"), (383, "100_2"), (441, "100_3")]:
            ## 알파벳 클릭
            pyautogui.moveTo(x= _x, y=130)
            pyautogui.click()
            time.sleep(8)
            
            
            ## 종목에 위치 & 우클릭
            pyautogui.moveTo(x=276, y=320)
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 엑셀로 보내기 클릭
            pyautogui.moveTo(x=391, y=560)
            pyautogui.click()
            time.sleep(1)
            
            ## csv 클릭
            pyautogui.moveTo(x=596, y=614)
            pyautogui.click()
            time.sleep(1)
            
            
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'증거금률{_rate}%종목신한_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            time.sleep(2.5)
            
            ## 문서 클릭
            img_capture = imglocation(r"C:\Shinhan\AlphaHTS\문서아이콘.png", confidence = 0.9)
            pyautogui.moveTo(img_capture)
            pyautogui.click()
            time.sleep(1)
        
            ## 저장버튼 클릭
            img_capture = imglocation(r"C:\Shinhan\AlphaHTS\저장버튼.png", confidence = 0.9)
            pyautogui.moveTo(img_capture)
            pyautogui.click()
            time.sleep(2.5)
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
        
        ## 신용 클릭
        pyautogui.moveTo(x=90, y=134)
        pyautogui.click()
        time.sleep(10)
        
        for _x, _rate in [(163, "신용가능1"), (218, "신용가능2"), (275, "신용가능3"), (329, "신용가능4"), (383, "신용가능5"), (441, "신용불가")]:
            ## 알파벳 클릭
            pyautogui.moveTo(x= _x, y=130)
            pyautogui.click()
            time.sleep(8)
            
            
            ## 종목에 위치 & 우클릭
            pyautogui.moveTo(x=276, y=320)
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 엑셀로 보내기 클릭
            pyautogui.moveTo(x=391, y=560)
            pyautogui.click()
            time.sleep(1)
            
            ## csv 클릭
            pyautogui.moveTo(x=596, y=614)
            pyautogui.click()
            time.sleep(1)
            
            
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'{_rate}종목신한_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            time.sleep(2.5)
            
            ## 문서 클릭
            img_capture = imglocation(r"C:\Shinhan\AlphaHTS\문서아이콘.png", confidence = 0.9)
            pyautogui.moveTo(img_capture)
            pyautogui.click()
            time.sleep(1)
        
            ## 저장버튼 클릭
            img_capture = imglocation(r"C:\Shinhan\AlphaHTS\저장버튼.png", confidence = 0.9)
            pyautogui.moveTo(img_capture)
            pyautogui.click()
            time.sleep(2.5)    
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
                
    def name_definition(self):
        self.name = '신한'
        self.hts_name = "신한SOL"
        self.hts_activate_flag = False

####################################################################################################
class Daishin(Kiwoom):
    def __init__(self):
        super().__init__()

    def name_definition(self):
        self.name = '대신'
        self.hts_name = "대신증권"
        self.hts_activate_flag = False



    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\DAISHIN\STARTER\ncStarter.exe'], stdin=PIPE)
        time.sleep(20)

        ## 입력칸 선택
        pyautogui.moveTo(x=639, y=455)
        pyautogui.click()
        time.sleep(1)

        ## 더블클릭 실행
        pyautogui.doubleClick()
        time.sleep(5)        
        press_keyboard(self.hts_info[self.name]['id'])
        time.sleep(5)
        press_keyboard(self.hts_info[self.name]['password'])
        time.sleep(5)
        pyautogui.press('enter')
        time.sleep(30)
        pyautogui.press('enter')
        time.sleep(3)
        pyautogui.press('esc')
        time.sleep(5)

    def download_file(self):
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=98, y=100)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('7248')
        pyautogui.press('enter')
        time.sleep(5)

        ## 증거금률로 커서 이동
        pyautogui.moveTo(x=138, y=244)
        pyautogui.click()
        time.sleep(5)

        ## 전체조회로 커서 이동
        pyautogui.moveTo(x=938, y=243)
        pyautogui.click(clicks=3, interval=1)
        time.sleep(5)

        ## 액셀로 내보내기로 커서 이동
        pyautogui.moveTo(x=1032, y=241)
        pyautogui.click()
        time.sleep(5)
        pyautogui.moveTo(x=1084, y=269)
        pyautogui.click()
        time.sleep(10)
        
        pyautogui.press('left')
        pyautogui.press('enter')
        time.sleep(10)
        pyautogui.moveTo(x=35, y=52)
        pyautogui.click()
        time.sleep(0.5)
        pyautogui.moveTo(x=66, y=279)
        pyautogui.click()
        time.sleep(0.5)        
        pyautogui.moveTo(x=658, y=315)
        pyautogui.click()
        time.sleep(0.5)
        pyautogui.moveTo(x=620, y=443)
        pyautogui.click()
        time.sleep(0.5)
        pyautogui.moveTo(x=614, y=474)
        pyautogui.click()
        time.sleep(0.5)
        pyautogui.moveTo(x=337, y=421)
        pyautogui.click()
        time.sleep(1)



        ## 파일 이름 입력
        today = datetime.today()
        file_name = f'증거금대신_{today.year}.{today.month}.{today.day}'
        pyperclip.copy(file_name)
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')
        time.sleep(2.5)
        
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".xlsx"
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()

        pyautogui.moveTo(x=755, y=593)
        pyautogui.click()
        time.sleep(5)
        pyautogui.moveTo(x=1887, y=21)
        pyautogui.click()


#########################################################################################################


class Samsung(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\삼성증권\POPHTS\POPHTS.exe'], stdin=PIPE)
        time.sleep(5)
        press_keyboard(self.hts_info[self.name]['id'])
        pyautogui.press('tab')
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
        time.sleep(5)
        press_keyboard('3131')
        pyautogui.press('enter')
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\삼성증권\POPHTS\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(1)
        time.sleep(10)
        
    def download_file(self):
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=86, y=94)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('1503')
        pyautogui.press('enter')
        time.sleep(5)
        
        ## 입력칸 선택
        pyautogui.moveTo(x=267, y=308)
        pyautogui.click()
        time.sleep(1)
        
        ## 증거금 입력
        pyperclip.copy('증거금')
        pyautogui.hotkey("ctrl", "v")
        time.sleep(2)
        pyautogui.press('enter')
        time.sleep(5)
        
        ## 증거금률로 커서 이동
        img_capture = imglocation(r"C:\삼성증권\POPHTS\증거금률.png")
        pyautogui.moveTo(img_capture)
        pyautogui.click()
        time.sleep(1)
    
        
        ## 더블클릭 실행
        pyautogui.doubleClick()
        time.sleep(2)
        
        for _y, _rate in [(612, 20), (628, 30), (643, 40), (660, 100)]:
            ## 알파벳 선택
            pyautogui.moveTo(x=631, y=328)
            pyautogui.click()
            time.sleep(1)
            
            ## 화살표 선택
            pyautogui.moveTo(x=614, y=596)
            pyautogui.click()
            time.sleep(1)
            
            ## 증거금 선택
            pyautogui.moveTo(x=566, y=_y)
            pyautogui.click()
            time.sleep(1)
        
            ## 검색 클릭
            pyautogui.moveTo(x=503, y=701)
            pyautogui.click()
            time.sleep(1)
        
            for _ in range(10):
                try:
                    img_capture = imglocation(r"C:\삼성증권\POPHTS\진행상태.png")
                    if img_capture != None: 
                        time.sleep(2)
                        break
                except:
                    pass
                time.sleep(2)
            
            ## 종목전송 클릭
            pyautogui.moveTo(x=1148, y=704)
            pyautogui.click()
            time.sleep(1)
        
            ## EXCEL 클릭
            pyautogui.moveTo(x=1263, y=815)
            pyautogui.click()
            time.sleep(1)
        
            for _ in range(100):
                try:
                    img_capture = imglocation(r"C:\삼성증권\POPHTS\excel_logo.png")
                    if img_capture != None: 
                        excel_x = img_capture[0]
                        excel_y = img_capture[1]
                        break
                except:
                    pass
        
            ## 작업표시줄에서 EXCEL 클릭
            pyautogui.moveTo(x=excel_x, y=excel_y)
            pyautogui.click()
            time.sleep(1)
        
        
            ## excel 저장하기
            pyautogui.hotkey("ctrl", "s")
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
        
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'증거금{_rate}%종목삼성_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)

            ## excel 닫기
            for _ in range(10):
                try:
                    img_capture = imglocation(r"C:\삼성증권\POPHTS\excel_x.png", confidence = 0.9)
                    pyautogui.moveTo(img_capture)
                    pyautogui.click()
                    time.sleep(2)
                    break
                except:
                    pass
            
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".xlsx"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
            
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=86, y=94)
        pyautogui.click()
        time.sleep(1)
        
        ## 화면번호 입력
        press_keyboard('1503')
        pyautogui.press('enter')
        time.sleep(5)
        
        ## 입력칸 선택
        pyautogui.moveTo(x=267, y=308)
        pyautogui.click()
        time.sleep(1)
        
        ## 신용가능 입력
        pyperclip.copy('신용가능')
        pyautogui.hotkey("ctrl", "v")
        time.sleep(2)
        pyautogui.press('enter')
        time.sleep(5)
        
        ## 신용가능종목으로 커서 이동
        img_capture = imglocation(r"C:\삼성증권\POPHTS\신용가능종목.png")
        pyautogui.moveTo(img_capture)
        pyautogui.click()
        time.sleep(1)
    
        
        ## 더블클릭 실행
        pyautogui.doubleClick()
        time.sleep(2)
        
        for _y, _flag in [(615, '신용가능'), (629, '신용불가능')]:
            ## 알파벳 선택
            pyautogui.moveTo(x=631, y=328)
            pyautogui.click()
            time.sleep(1)
            
            ## 화살표 선택
            pyautogui.moveTo(x=600, y=594)
            pyautogui.click()
            time.sleep(1)
            
            ## 신용가능 선택
            pyautogui.moveTo(x=542, y=_y)
            pyautogui.click()
            time.sleep(1)
            
            ## 검색 클릭
            pyautogui.moveTo(x=503, y=701)
            pyautogui.click()
            time.sleep(1)
        
            for _ in range(10):
                try:
                    img_capture = imglocation(r"C:\삼성증권\POPHTS\진행상태.png")
                    if img_capture != None: 
                        time.sleep(2)
                        break
                except:
                    pass
                time.sleep(2)
            
            ## 종목전송 클릭
            pyautogui.moveTo(x=1148, y=704)
            pyautogui.click()
            time.sleep(1)
        
            ## EXCEL 클릭
            pyautogui.moveTo(x=1263, y=815)
            pyautogui.click()
            time.sleep(1)
        
            for _ in range(100):
                try:
                    img_capture = imglocation(r"C:\삼성증권\POPHTS\excel_logo.png")
                    if img_capture != None: 
                        excel_x = img_capture[0]
                        excel_y = img_capture[1]
                        break
                except:
                    pass
        
            ## 작업표시줄에서 EXCEL 클릭
            pyautogui.moveTo(x=excel_x, y=excel_y)
            pyautogui.click()
            time.sleep(1)
        
        
            ## excel 저장하기
            pyautogui.hotkey("ctrl", "s")
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
            pyautogui.press('enter')
            time.sleep(1)
        
            ## 파일 이름 입력
            today = datetime.today()
            file_name = f'{_flag}종목삼성_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)
            
            ## excel 닫기
            img_capture = imglocation(r"C:\삼성증권\POPHTS\excel_x.png", confidence = 0.9)
            pyautogui.moveTo(img_capture)
            pyautogui.click()
            time.sleep(2)    
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".xlsx"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
                
    def name_definition(self):
        self.name = '삼성'
        self.hts_name = "POP HTS"  

class Hankook(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r"C:\eFriend Plus\efriendplus\efriendPlus.exe"], stdin=PIPE)
        time.sleep(5)

        for i in range(100):
            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\로그인창_아이콘.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass

            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\로그인창_아이콘2.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass
            time.sleep(2.5)
    
        pyautogui.moveTo(img_capture)
        pyautogui.doubleClick()
        time.sleep(3)
    
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\시세조회 확인.png")
                if img_capture != None:
                    break
            except:
                pass
            time.sleep(1)
    
        press_keyboard(self.hts_info[self.name]['id'])
        pyautogui.press('tab')
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
        
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\시세전용 로그인 화면.png")
                if img_capture != None:
                    pyautogui.press('enter')
                    break
            except:
                pass
            time.sleep(1)
                
        for i in range(100):
            ## hts 활성화 상태(커서 누른 후)
            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            ## hts 비활성화 상태
            try:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\hts_logo2.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(2)
            
        time.sleep(5)
        
    def download_file(self):
        ## 고객확인의무 창 뜨면 ESC 클릭
        try:
            img_capture = imglocation(r"C:\eFriend Plus\efriendplus\고객확인의무.png")
            if img_capture!= None:
                img_capture = imglocation(r"C:\eFriend Plus\efriendplus\고객확인의무닫기.png")
                pyautogui.moveTo(img_capture)
                pyautogui.click()
                time.sleep(1)
        except:
            pass
        
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=80, y=75)
        pyautogui.click()
        time.sleep(1)
        
        ## 모든 화면 닫기 입력
        press_keyboard('9230')
        pyautogui.press('enter')
        time.sleep(1)
        
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=80, y=75)
        pyautogui.click()
        time.sleep(1)
    
        ## 화면번호 입력
        press_keyboard('0478')
        pyautogui.press('enter')
        time.sleep(5)
    
        ## 증거금률별 종목정보 파일(csv) 저장
        for _x, _y, _rate in [(304, 199, 20), (304, 215, 30), (304, 234, 40), (304, 251, 50), (304, 267, 60), (304, 284, 100)]: # 20%, 30%, 40%, 50%, 60%, 100%
            ## 증거금구분 펼치기
            pyautogui.moveTo(x=331, y=175)
            pyautogui.click()
            time.sleep(1)
    
            ## 증거금구분(20, 30, 40, 50, 60, 100) 선택
            pyautogui.moveTo(x=_x, y=_y)
            pyautogui.click()
            time.sleep(4)
    
            ## 종목에 커서 두기
            pyautogui.moveTo(x=624, y=384)
    
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            count = 0
            pre_img = None
            while True:
                img = ImageGrab.grab((414, 535,808, 662))
                if pre_img != None:
                    if pre_img == img:
                        count += 1
                        ## 세번 중복되면 break
                        if count == 20:
                            break
                    else:
                        count = 0
    
                pyautogui.scroll(-500)
                time.sleep(0.2)
                pre_img = img
    
            ## 오른쪽 클릭
            pyautogui.rightClick()
            time.sleep(5)
    
            ## 파일로 보내기에 커서 두기
            pyautogui.moveTo(x=780, y=941)
            pyautogui.click()
            time.sleep(2.5)
    
            ## csv로 저장 클릭
            pyautogui.moveTo(x=972, y=1027)
            pyautogui.click()
            time.sleep(2.5)
    
            ## 파일명 지정 및 저장
            today = datetime.today()
            file_name = f'증거금{_rate}%종목한투_{today.year}.{today.month}.{today.day}'
            pyperclip.copy(file_name)
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
            time.sleep(2.5)
            real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
            if os.path.isfile(real_path):
                print(file_name + "<--Saved!!")
            else:
                print(file_name + "<--Error!!")
                create_error()
                break
    
    
        ## 신용가능종목 정보 파일(csv) 저장
        ## 화면번호 입력칸 선택
        pyautogui.moveTo(x=80, y=75)
        pyautogui.click()
        time.sleep(1)
    
        ## 화면번호 입력
        press_keyboard('0477')
        pyautogui.press('enter')
        time.sleep(5)
    
        ## 종목에 커서 두기
        pyautogui.moveTo(x=751, y=547)
    
        ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
        count = 0
        pre_img = None
        while True:
            img = ImageGrab.grab((414, 535,808, 662))
            if pre_img != None:
                if pre_img == img:
                    count += 1
                    ## 세번 중복되면 break
                    if count == 20:
                        break
                else:
                    count = 0
    
            pyautogui.scroll(-500)
            time.sleep(0.2)
            pre_img = img
    
        ## 오른쪽 클릭
        pyautogui.rightClick()
        time.sleep(5)
    
        ## 파일로 보내기에 커서 두기
        pyautogui.moveTo(x=886, y=957)
        pyautogui.click()
        time.sleep(2.5)
    
        ## csv로 저장 클릭
        pyautogui.moveTo(x=1100, y=929)
        pyautogui.click()
        time.sleep(2.5)
    
        ## 파일명 지정 및 저장
        today = datetime.today()
        file_name = f'신용가능종목한투_{today.year}.{today.month}.{today.day}'
        pyperclip.copy(file_name)
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')  
        time.sleep(2.5)
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()
        
        ## 신용주문불가 클릭
        img_capture = imglocation(r"C:\eFriend Plus\efriendplus\신용주문불가.png")
        pyautogui.moveTo(img_capture)
        pyautogui.click()
        time.sleep(5)
    
        ## 종목에 커서 두기
        pyautogui.moveTo(x=751, y=547)
    
        ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
        count = 0
        pre_img = None
        while True:
            img = ImageGrab.grab((414, 535,808, 662))
            if pre_img != None:
                if pre_img == img:
                    count += 1
                    ## 세번 중복되면 break
                    if count == 20:
                        break
                else:
                    count = 0
    
            pyautogui.scroll(-500)
            time.sleep(0.2)
            pre_img = img
    
        ## 오른쪽 클릭
        pyautogui.rightClick()
        time.sleep(5)
    
        ## 파일로 보내기에 커서 두기
        pyautogui.moveTo(x=886, y=957)
        time.sleep(2.5)
    
        ## csv로 저장 클릭
        pyautogui.moveTo(x=1100, y=929)
        pyautogui.click()
        time.sleep(2.5)
    
        ## 파일명 지정 및 저장
        today = datetime.today()
        file_name = f'신용불가종목한투_{today.year}.{today.month}.{today.day}'
        pyperclip.copy(file_name)
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')   
        time.sleep(2.5)
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name + ".csv"
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()

    def name_definition(self):
        self.name = '한투'
        self.hts_name = "eFriend Plus"  

class Hana(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\1QHTS\HFStarter.exe'], stdin=PIPE)
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\1QHTS\image\login.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass
            time.sleep(1)
        
        time.sleep(5)
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
        pyautogui.press('enter')
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\1QHTS\image\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(1)
        time.sleep(10)
        
    def download_file(self):
        # 증거금률 조회 화면번호
        pyautogui.moveTo(x=99, y=66)
        pyautogui.click()
        time.sleep(1)
        press_keyboard("7355")
        time.sleep(3)

        ## 증거금률별 종목정보 파일(csv) 저장
        for _x, _y, _rate, _grd in [(156, 205, 30, 'A'), (493, 205, 30, 'B'), (833, 205, 40, 'C'), (1168, 205, 100, 'D'), (1516, 205, 100, 'E')]: 
            pyautogui.moveTo(x=_x, y=_y)
            pyautogui.click()
            time.sleep(1)
        
            pyautogui.moveTo(x=1843, y=210)
            
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            
            pre_img = None
            count = 0
            while True:
                img = ImageGrab.grab((144, 383, 443, 767))
                if pre_img == img:
                    count += 1
                    if count == 20:
                        break
                else:
                    count = 0
            
                pyautogui.click()
                time.sleep(0.4)
                pre_img = img
            
            pyautogui.moveTo(x=888, y=420)
            pyautogui.rightClick()
            time.sleep(1)
            pyautogui.moveTo(x=981, y=812)
            pyautogui.click()
            time.sleep(1)
        
            ## 파일명 지정 및 저장
            today = datetime.today()
            pyperclip.copy(f'증거금{_rate}%' + _grd + f'종목하나_{today.year}.{today.month}.{today.day}')
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')     
        
        time.sleep(10)
        os.chdir(r"C:\Users\kiwoom\Documents")
        out = []
        file_count = 0
        for file in glob.glob(f'*종목하나_{today.year}.{today.month}.{today.day}.csv'):
            pre_result = pd.read_csv(file, encoding = 'cp949')
            out.append(pre_result)
            file_count += 1
    
        if file_count != 5:
            print('file count error!!')
            create_error()
        
        out = pd.concat(out, axis = 0)
        out.to_excel(f'최종증거금률하나_{today.year}.{today.month}.{today.day}.xlsx', index=False)
        
        out2 = out[out['신용대출가능여부'] == '신용대출가능']
        
        out2.to_excel(f'신용가능종목하나_{today.year}.{today.month}.{today.day}.xlsx', index=False)
    
        file_name = f'최종증거금률하나_{today.year}.{today.month}.{today.day}.xlsx'
        file_name1 = f'신용가능종목하나_{today.year}.{today.month}.{today.day}.xlsx'
        
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name
        real_path1 = r"C:\Users\kiwoom\Documents" + "\\" + file_name1
    
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()
        
        if os.path.isfile(real_path1):
            print(file_name1 + "<--Saved!!")
        else:
            print(file_name1 + "<--Error!!")   
            create_error()

    def name_definition(self):
        self.name = '하나'
        self.hts_name = "1Q HTS"           

class Meritz(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\메리츠증권\iMERITZ XII\Main\imeritz.exe'], stdin=PIPE)
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\메리츠증권\login2.png", confidence = 0.9)
                if img_capture != None:
                    pyautogui.moveTo(img_capture)
                    pyautogui.click()
                    break
            except:
                pass
            time.sleep(1)

        for i in range(100):
            try:
                img_capture = imglocation(r"C:\메리츠증권\시세전용 확인.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass
            time.sleep(1)
                        
        time.sleep(5)
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
        time.sleep(10)
        pyautogui.press('enter')
        time.sleep(4)
                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\메리츠증권\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(1)
        time.sleep(10)

        
    def download_file(self):
        # 증거금률 조회 화면번호
        pyautogui.moveTo(x=54, y=65)
        pyautogui.click()
        time.sleep(1)
        press_keyboard("1148")
        time.sleep(5)



        pyautogui.moveTo(x=836, y=259)
        pyautogui.click()  
        time.sleep(10)

        ## 선택영역 이미지가 바뀌지 않을때까지 스크롤

        pre_img = None
        count = 0
        while True:
            img = ImageGrab.grab((41, 306, 290, 758))
            if pre_img == img:
                count += 1
                if count == 10:
                    break
            else:
                count = 0
            
        pyautogui.click()
        time.sleep(1.5)
        pre_img = img

        pyautogui.moveTo(x=690, y=476)
        pyautogui.rightClick()
        time.sleep(1)
        pyautogui.moveTo(x=868, y=646)
        pyautogui.click()
        time.sleep(1)
        pyautogui.moveTo(x=1007, y=780)
        pyautogui.click()

        ## 파일명 지정 및 저장
        today = datetime.today()
        time.sleep(1)
        pyperclip.copy(f'증거금메리츠_{today.year}.{today.month}.{today.day}')
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')


    def name_definition(self):
        self.name = '메리츠'
        self.hts_name = "iMeritz"      
       
class Mirae(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\미래에셋증권\카이로스\kairos.exe'], stdin=PIPE)

        for i in range(100):
            try:
                img_capture = imglocation(r"C:\미래에셋증권\login2.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass
    
            time.sleep(1)
                    
        time.sleep(3)

        press_keyboard(self.hts_info[self.name]['id'])
        pyautogui.press('tab')
        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')
        time.sleep(1)
        pyautogui.press('enter')

        for i in range(100):
            try:
                img_capture = imglocation(r"C:\미래에셋증권\hts_logo.png")
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(1)
        time.sleep(10)

###################################################################################################################################

    def download_file(self):
        # 증거금률 조회 화면번호
        pyautogui.moveTo(x=59, y=66)
        pyautogui.click()
        time.sleep(2)
        press_keyboard("0166")
        time.sleep(3)
        
        for _x, _y, _rate in [(760, 366, 20), (786, 366, 30), (836, 370, 40), (895, 368, 100)]: # 20%, 30%, 40%, 100%
            ## 전체 클릭
            #pyautogui.moveTo(x=155, y=319)
            #pyautogui.click()
            #time.sleep(1)
            
            ## 20% 클릭
            pyautogui.moveTo(x=_x, y=_y)
            pyautogui.click()
            time.sleep(1)
            
            ## 종목 중간에 커서 위치
            pyautogui.moveTo(x=946, y=531)
            time.sleep(1)
            
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            pre_img = None
            count = 0
            while True:
                img = ImageGrab.grab((722, 632, 836, 763))
                if pre_img == img:
                    count += 1
                    if count == 2:
                        break
                else:
                    count = 0
                pyautogui.scroll(-250)
                time.sleep(0.6)
                pre_img = img

            ## 오른쪽 클릭
            pyautogui.rightClick()
            time.sleep(1)
            
            ## 엑셀로 저장 선택
            pyautogui.moveTo(x=1050, y=879)
            pyautogui.click()
            time.sleep(1)

            ## 파일명 지정 및 저장
            today = datetime.today()
            pyperclip.copy(f'증거금{_rate}%종목미래_{today.year}.{today.month}.{today.day}')
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')

            
        
        pyautogui.moveTo(x=59, y=66)
        pyautogui.click()
        time.sleep(2)
        press_keyboard("0917")
        time.sleep(3)
        
        pyautogui.moveTo(x=532, y=394)
        time.sleep(4)
        
        ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
        
        pre_img = None
        count = 0
        while True:
            img = ImageGrab.grab((404, 348, 702, 840))
            if pre_img == img:
                count += 1
                if count == 20:
                    break
            else:
                count = 0
        
            pyautogui.scroll(-500)
            time.sleep(0.2)
            pre_img = img
        
        
        ## 엑셀파일로 내보내기 위해 제일 위의 종목위에 마우스 두기
        pyautogui.moveTo(x=515, y=373)
        pyautogui.rightClick()
        time.sleep(1)
        
        ## Excel파일로 저장 클릭
        pyautogui.moveTo(x=728, y=726)
        pyautogui.click()
        time.sleep(1)
        
        ## 파일명 지정 및 저장
        today = datetime.today()
        pyperclip.copy(f'신용가능종목미래_{today.year}.{today.month}.{today.day}')
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')
        
        os.chdir(r"C:\Users\kiwoom\Documents")
        
        out = []
        for _rate in [20, 30, 40, 100]:
            result = pd.read_csv(f"증거금{_rate}%종목미래_{today.year}.{today.month}.{today.day}.csv", encoding = 'cp949')
            result['미래'] = _rate
            out.append(result[['종목명', '미래']])
        
        out = pd.concat(out, axis = 0)
        out.to_excel(f"최종증거금률미래_{today.year}.{today.month}.{today.day}.xlsx", index=False)
    
        file_name = f'최종증거금률미래_{today.year}.{today.month}.{today.day}.xlsx'
        file_name1 = f'신용가능종목미래_{today.year}.{today.month}.{today.day}.csv'
        
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name
        real_path1 = r"C:\Users\kiwoom\Documents" + "\\" + file_name1
    
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()
        
        if os.path.isfile(real_path1):
            print(file_name1 + "<--Saved!!")
        else:
            print(file_name1 + "<--Error!!")
            create_error()

    def name_definition(self):
        self.name = '미래'
        self.hts_name = "미래에셋증권"  
 
class NH(Kiwoom):
    def __init__(self):
        super().__init__()

    def login(self):
        pyautogui.hotkey('super', 'd')
        proc = Popen([r'C:\Users\kiwoom\Desktop\WMV\QV\wmtslogin.exe'], stdin=PIPE)
        
        while True:
            try:
                img_capture = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\login2.png", confidence = 0.9)
                if img_capture != None:
                    break
            except:
                pass
            time.sleep(1)
            
            try:
                img_capture_logo = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\nh_logo2.png", confidence = 0.9)
                if img_capture_logo != None:
                    pyautogui.moveTo(x=img_capture_logo[0], y=img_capture_logo[1])
                    pyautogui.doubleClick()
                    break
            except:
                pass
            time.sleep(1)
                    
        time.sleep(3)

        press_keyboard(self.hts_info[self.name]['password'])
        pyautogui.press('enter')

                
        for i in range(100):
            try:
                img_capture = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\hts_logo.png", confidence = 0.9)
                if img_capture!= None:    
                    download_flag = 1
                    break
            except:
                pass
            
            time.sleep(1)
        time.sleep(10)
        
    def download_file(self):
        ## 모든 화면 닫기 입력
        pyautogui.moveTo(x=216, y=76)
        pyautogui.click()
        time.sleep(1)
        press_keyboard("0027")
        time.sleep(3)
        
        pyautogui.click()
        time.sleep(1)
        press_keyboard("1950")
        
        pyautogui.moveTo(x=108, y=386)
        pyautogui.doubleClick()
        time.sleep(1)
        
        pyautogui.moveTo(x=129, y=521)
        pyautogui.doubleClick()
        time.sleep(1)
    

        ## 증거금률별 종목정보 파일(csv) 저장
        for _x, _y, _rate in [(557, 306, 30), (682, 310, 40), (813, 306, 100)]: # 30%, 40%, 100%
            ## 증거금구분(30, 40, 100) 선택
            pyautogui.moveTo(x=_x, y=_y)
            pyautogui.click()
            time.sleep(1)
        
            pyautogui.moveTo(x=1159, y=695)
            pyautogui.click()
            time.sleep(1)    
        
            pyautogui.moveTo(x=744, y=791)
            time.sleep(1)
            
            ## 선택영역 이미지가 바뀌지 않을때까지 스크롤
            
            pre_img = None
            count = 0
            while True:
                img = ImageGrab.grab((307, 725, 456, 937))
                if pre_img == img:
                    count += 1
                    if count == 20:
                        break
                else:
                    count = 0
            
                pyautogui.scroll(-500)
                time.sleep(0.2)
                pre_img = img
        
            
            ## 엑셀파일로 내보내기 위해 전송 버튼
            pyautogui.moveTo(x=650, y=693)
            pyautogui.click()
            time.sleep(1)
            
            ## Excel파일로 저장 클릭
            pyautogui.moveTo(x=730, y=813)
            pyautogui.click()
            time.sleep(1)
        
            ## 파일명 지정 및 저장
            today = datetime.today()
            pyperclip.copy(f'증거금{_rate}%종목NH_{today.year}.{today.month}.{today.day}')
            pyautogui.hotkey("ctrl", "v")
            pyautogui.press('enter')
    
            
        pyautogui.moveTo(x=216, y=76)
        pyautogui.click()
        time.sleep(1)
        press_keyboard("8134")
        time.sleep(3)
    
        for i in range(10):
            try:
                img_capture_신용조회 = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\nh_신용종목조회.png", confidence = 0.9)    
                pyautogui.moveTo(x=img_capture_신용조회[0]+100, y=img_capture_신용조회[1]+20)
                time.sleep(3)
                break
            except:
                pass
        
        ## 선택영역 이미지가 바뀌지 않을때까지 클릭
        for _ in range(10):
            try:
                img_capture_선택영역 = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\nh_신용종목이미지.png", confidence = 0.7)    
                break
            except:
                pass
            time.sleep(1)
        
        pre_img = None
        count = 0
        while True:
            img = ImageGrab.grab((img_capture_선택영역[0], img_capture_선택영역[1], img_capture_선택영역[0] + 200, img_capture_선택영역[1] + 460))
            if pre_img == img:
                count += 1
                if count == 20:
                    break
            else:
                count = 0
        
            pyautogui.click()
            time.sleep(0.4)
            pre_img = img
        
        
        ## 엑셀파일로 내보내기 위해 제일 위의 종목위에 마우스 두기
        for i in range(10):
            try:
                img_capture_tofile = imglocation(r"C:\Users\kiwoom\Desktop\WMV\QV\nh_신용종목캡처.png", confidence = 0.7) 
                if img_capture_tofile != None:
                    break
            except:
                time.sleep(1)
                
        pyautogui.moveTo(x=img_capture_tofile[0], y=img_capture_tofile[1] + 100)
        pyautogui.rightClick()
        time.sleep(1)
        
        ## Excel파일로 저장 클릭
        pyautogui.moveTo(x=img_capture_tofile[0] + 205, y=img_capture_tofile[1] + 370)
        pyautogui.click()
        time.sleep(1)
        
        ## 파일명 지정 및 저장
        today = datetime.today()
        pyperclip.copy(f'신용가능종목NH_{today.year}.{today.month}.{today.day}')
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press('enter')
    
        os.chdir(r"C:\Users\kiwoom\Documents")
        
        out = []
        for _rate in [30, 40, 100]:
            result = pd.read_excel(f"증거금{_rate}%종목NH_{today.year}.{today.month}.{today.day}.xlsx", engine='openpyxl')
            result['NH'] = _rate
            out.append(result[['종목명', 'NH']])
        
        result_신용 = pd.read_csv(f'신용가능종목NH_{today.year}.{today.month}.{today.day}.csv', encoding = 'cp949')
        result_신용_final = result_신용[result_신용.iloc[:,2] == '대출가능/신용가능']
        result_신용_final.to_excel(f'신용가능종목NH_{today.year}.{today.month}.{today.day}.xlsx', index=False)
        
        out = pd.concat(out, axis = 0)
        out.to_excel(f"최종증거금률NH_{today.year}.{today.month}.{today.day}.xlsx", index=False)
    
        
        file_name = f'최종증거금률NH_{today.year}.{today.month}.{today.day}.xlsx'
        file_name1 = f'신용가능종목NH_{today.year}.{today.month}.{today.day}.xlsx'
        
        real_path = r"C:\Users\kiwoom\Documents" + "\\" + file_name
        real_path1 = r"C:\Users\kiwoom\Documents" + "\\" + file_name1
    
        if os.path.isfile(real_path):
            print(file_name + "<--Saved!!")
        else:
            print(file_name + "<--Error!!")
            create_error()
        
        if os.path.isfile(real_path1):
            print(file_name1 + "<--Saved!!")
        else:
            print(file_name1 + "<--Error!!")
            create_error()

    def name_definition(self):
        self.name = 'NH'
        self.hts_name = "QV MASTER"   
         
def total_output_save():
    os.chdir(r"C:\Users\kiwoom\Documents")
    
    file_list = os.listdir("./")
    today = datetime.today()
    target_word = f'{today.year}.{today.month}.{today.day}'
    target_file = []
    for file in file_list:
        if target_word in file:
            target_file.append(file)
    
    
    def kiwoom_신용구분(x):
        if len(x) == 1:
            return '불가'
        else:
            return x[1].split('/')[0]
    
    def kiwoom_담보구분(x):
        if len(x) == 1:
            return '불가'
        else:
            return x[1].split('/')[0]

    def clean_kb_종목명(x):
        if x[-3:] == '(F)':
            return x[:-3]
        else:
            return x
    
    def clean_kiwoom(result, 증거금률):
        out = result[['종목번호', '종목명', '구분']]
        if 증거금률 == '100':
            out['증거금률'] = '100'
            out['신용가능여부'] = '불가'
            out['대출가능여부'] = '불가'
        else:
            out['증거금률'] = 증거금률
            out['신용가능여부'] = out['구분'].str.split('신용').apply(lambda x : kiwoom_신용구분(x))
            out['대출가능여부'] = out['구분'].str.split('담보').apply(lambda x : kiwoom_담보구분(x))
        out['종목번호'] = out['종목번호'].str.replace("'", 'A')
        return out[['종목번호', '종목명', '증거금률', '신용가능여부', '대출가능여부']]
    
    def clean_미래_증거금(result, 증거금률):
        out = result[['종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목명', '증거금률']]
    
    def clean_삼성_증거금(result, 증거금률):
        out = result[['종목코드', '종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목코드', '종목명', '증거금률']]
    
    def clean_하나_증거금(result, 증거금률):
        out = result[['종목코드', '종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목코드', '종목명', '증거금률']]
    
    def clean_한투_증거금(result, 증거금률):
        out = result[['종목코드', '종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목코드', '종목명', '증거금률']]
    
    def clean_신한_증거금(result, 증거금률):
        out = result[['종목코드', '종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목코드', '종목명', '증거금률']]
    
    def clean_NH_증거금(result, 증거금률):
        out = result[['종목명']].copy()
        out['증거금률'] = 증거금률
        return out[['종목명', '증거금률']]
    
    def clean_메리츠_증거금신용대출(result):
        out = result[['종목코드', '종목명', '증거금율', '신용가능여부', '대출가능여부']].copy()
        out['증거금률'] = out['증거금율']

        out['신용가능여부_'] = '불가'
        out.loc[out['신용가능여부'] == 'Y', '신용가능여부_'] = '가능'
        
        out['대출가능여부_'] = '불가'
        out.loc[out['대출가능여부'] == 'Y', '대출가능여부_'] = '가능'
        return out[['종목코드', '종목명', '증거금률', '신용가능여부_', '대출가능여부_']]

    def clean_대신_증거금신용대출(result):
        out = result[['종목코드', '종목명', '현금 증거금', '신용가능', '대출가능']].copy()
        out['증거금률'] = out['현금 증거금']

        out['신용가능_'] = '불가'
        out.loc[out['신용가능'] == '가 능', '신용가능_'] = '가능'
        
        out['대출가능_'] = '불가'
        out.loc[out['대출가능'] == '가 능', '대출가능_'] = '가능'
        return out[['종목코드', '종목명', '증거금률', '신용가능_', '대출가능_']]
    
    def clean_kb_증거금(result, 증거금률):
        if 증거금률 == '100':
            out = result[['종목번호', '종목명']].copy()
            out['증거금률'] = '100'
        else:
            out = result[['종목번호', '종목명', '증거금율']].copy()
            out['증거금률'] = out['증거금율'].apply(lambda x : x[2:])
        return out[['종목번호', '종목명', '증거금률']]
    
    ###################################################################
    
    def clean_미래_신용(result, 가능불가):
        out = result[['종목명']].copy()
        out['신용가능여부'] = 가능불가
        return out[['종목명', '신용가능여부']]
    
    def clean_삼성_신용(result, 가능불가):
        out = result[['종목코드', '종목명']].copy()
        out['신용가능여부'] = 가능불가
        return out[['종목코드', '종목명', '신용가능여부']]
    
    def clean_하나_신용(result, 가능불가):
        out = result[['종목코드', '종목명']].copy()
        out['신용가능여부'] = 가능불가
        return out[['종목코드', '종목명', '신용가능여부']]
    
    def clean_한투_신용(result, 가능불가):
        out = result[['코드', '종목명']].copy()
        out['신용가능여부'] = 가능불가
        out['종목코드'] = out['코드']
        return out[['종목코드', '종목명', '신용가능여부']]
    
    def clean_신한_신용(result, 가능불가):
        out = result[['종목코드', '종목명']].copy()
        out['신용가능여부'] = 가능불가
        return out[['종목코드', '종목명', '신용가능여부']]
    
    def clean_NH_신용대출(result):
        out = result[['종목코드', '종목명', '대출/신용\r\n가능여부']].copy()
        out['신용가능여부'] = out['대출/신용\r\n가능여부'].str.split('/').apply(lambda x:x[1].replace('신용', ''))
        out['대출가능여부'] = out['대출/신용\r\n가능여부'].str.split('/').apply(lambda x:x[0].replace('대출', ''))
        return out[['종목코드', '종목명', '신용가능여부', '대출가능여부']]
    
    def clean_kb_신용(result, 가능불가):
        out = result[['종목명']].copy()
        out['종목명'] = out['종목명'].apply(lambda x:clean_kb_종목명(x))
        out['신용가능여부'] = 가능불가
        return out[['종목명', '신용가능여부']]
    
    def clean_kb_대출(result):
        out = result[['종목명']].copy()
        out['종목명'] = out['종목명'].apply(lambda x:clean_kb_종목명(x))
        out['대출가능여부'] = '가능'
        return out[['종목명', '대출가능여부']]
        
    증권사_list = ['키움', '미래', '삼성', '하나', '한투', '신한', 'NH', '메리츠', 'kb', '대신']
    키움_output = []
    미래_증거금_output = []
    삼성_증거금_output = []
    하나_증거금_output = []
    한투_증거금_output = []
    신한_증거금_output = []
    NH_증거금_output = []
    메리츠_증거금_output = []
    kb_증거금_output = []
    대신_증거금_output = []
    
    미래_신용_output = []
    삼성_신용_output = []
    하나_신용_output = []
    한투_신용_output = []
    신한_신용_output = []
    NH_신용대출_output = []
    메리츠_신용대출_output = []
    kb_신용_output = []
    kb_대출_output = []
    대신_신용대출_output = []

    
    for s in 증권사_list:
        for t in target_file:
            if '최종증거금률' in t:continue
            if s in t:
                if '증거금' in t:
                    증거금률 = t.split('증거금')[1].split('종목')[0].split('%')[0].replace('률', '').split('_')[0]
                    try:
                        result = pd.read_csv(t, encoding = 'cp949')
                    except:
                        try:
                            result = pd.read_csv(t, encoding = 'utf8')
                        except:
                            result = pd.read_excel(t)
                    if 증거금률 not in 증권사_list:
                        if s == '키움':
                            키움_output.append(clean_kiwoom(result, 증거금률))
                        elif s == '미래':
                            미래_증거금_output.append(clean_미래_증거금(result, 증거금률))
                        elif s == '삼성':
                            삼성_증거금_output.append(clean_삼성_증거금(result, 증거금률))
                        elif s == '하나':
                            하나_증거금_output.append(clean_하나_증거금(result, 증거금률))
                        elif s == '한투':
                            한투_증거금_output.append(clean_한투_증거금(result, 증거금률))
                        elif s == '신한':
                            신한_증거금_output.append(clean_신한_증거금(result, 증거금률))
                        elif s == 'NH':
                            NH_증거금_output.append(clean_NH_증거금(result, 증거금률))
                        elif s == '메리츠':
                            pass
                        elif s == 'kb':
                            kb_증거금_output.append(clean_kb_증거금(result, 증거금률))

                    else:
                        if 증거금률 == '메리츠':
                            메리츠_증거금_output.append(clean_메리츠_증거금신용대출(result))
                        elif 증거금률 == '대신':
                            대신_증거금_output.append(clean_대신_증거금신용대출(result))
    
                elif '신용' in t:
                    가능불가 = t[2:4]
                    try:
                        result = pd.read_csv(t, encoding = 'cp949')
                    except:
                        try:
                            result = pd.read_csv(t, encoding = 'utf8')
                        except:
                            result = pd.read_excel(t)
                    if s == '미래':
                        미래_신용_output.append(clean_미래_신용(result, 가능불가))
                    elif s == '삼성':
                        삼성_신용_output.append(clean_삼성_신용(result, 가능불가))
                    elif s == '하나':
                        하나_신용_output.append(clean_하나_신용(result, 가능불가))
                    elif s == '한투':
                        한투_신용_output.append(clean_한투_신용(result, 가능불가))
                    elif s == '신한':
                        신한_신용_output.append(clean_신한_신용(result, 가능불가))
                    elif s == 'NH':
                        try:
                            NH_신용대출_output.append(clean_NH_신용대출(result))
                        except:
                            pass
                    elif s == '메리츠':
                        pass
                    elif s == 'kb':
                        kb_신용_output.append(clean_kb_신용(result, 가능불가))
                elif '주식대출' in t:
                    try:
                        result = pd.read_csv(t, encoding = 'cp949')
                    except:
                        try:
                            result = pd.read_csv(t, encoding = 'utf8')
                        except:
                            result = pd.read_excel(t)
                    if s == 'kb':
                        kb_대출_output.append(clean_kb_대출(result))
    
    키움_최종 = pd.concat(키움_output).drop_duplicates('종목명')
    키움_최종 = 키움_최종[['종목번호', '종목명', '증거금률', '신용가능여부', '대출가능여부']]
    키움_최종['증거금률'] = 키움_최종['증거금률'] + '%'
    키움_최종 = 키움_최종.rename(columns = {'종목번호' : '종목코드', '증거금률' : '키움_증거금률', '신용가능여부' : '키움_신용가능여부', 
                                          '대출가능여부' : '키움_대출가능여부'})
    
    미래_증거금_최종 = pd.concat(미래_증거금_output).drop_duplicates('종목명')
    미래_신용_최종 = pd.concat(미래_신용_output).drop_duplicates('종목명')
    미래_최종 = 미래_증거금_최종.merge(미래_신용_최종, how = 'left', on = '종목명').fillna('불가')
    미래_최종['대출가능여부'] = '-'
    미래_최종 = 미래_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    미래_최종['증거금률'] = 미래_최종['증거금률'] + '%'
    미래_최종 = 미래_최종.rename(columns = {'증거금률' : '미래_증거금률', '신용가능여부' : '미래_신용가능여부', '대출가능여부' : '미래_대출가능여부'})
    
    삼성_증거금_최종 = pd.concat(삼성_증거금_output).drop_duplicates('종목명')
    삼성_신용_최종 = pd.concat(삼성_신용_output).drop_duplicates('종목명')
    삼성_최종 = 삼성_증거금_최종.merge(삼성_신용_최종, how = 'left', on = '종목명').fillna('불가')
    삼성_최종['대출가능여부'] = '-'
    삼성_최종 = 삼성_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    삼성_최종['증거금률'] = 삼성_최종['증거금률'] + '%'
    삼성_최종 = 삼성_최종.rename(columns = {'증거금률' : '삼성_증거금률', '신용가능여부' : '삼성_신용가능여부', '대출가능여부' : '삼성_대출가능여부'})
    
    하나_증거금_최종 = pd.concat(하나_증거금_output).drop_duplicates('종목명')
    하나_신용_최종 = pd.concat(하나_신용_output).drop_duplicates('종목명')
    하나_최종 = 하나_증거금_최종.merge(하나_신용_최종, how = 'left', on = '종목명').fillna('불가')
    하나_최종['대출가능여부'] = '-'
    하나_최종 = 하나_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    하나_최종['증거금률'] = 하나_최종['증거금률'] + '%'
    하나_최종 = 하나_최종.rename(columns = {'증거금률' : '하나_증거금률', '신용가능여부' : '하나_신용가능여부', '대출가능여부' : '하나_대출가능여부'})
    
    한투_증거금_최종 = pd.concat(한투_증거금_output).drop_duplicates('종목명')
    한투_신용_최종 = pd.concat(한투_신용_output).drop_duplicates('종목명')
    한투_최종 = 한투_증거금_최종.merge(한투_신용_최종, how = 'left', on = '종목명').fillna('불가')
    한투_최종['대출가능여부'] = '-'
    한투_최종 = 한투_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    한투_최종['증거금률'] = 한투_최종['증거금률'] + '%'
    한투_최종 = 한투_최종.rename(columns = {'증거금률' : '한투_증거금률', '신용가능여부' : '한투_신용가능여부', '대출가능여부' : '한투_대출가능여부'})
    
    신한_증거금_최종 = pd.concat(신한_증거금_output).drop_duplicates('종목코드')
    신한_신용_최종 = pd.concat(신한_신용_output).drop_duplicates('종목코드')
    신한_최종 = 신한_증거금_최종.merge(신한_신용_최종, how = 'left', on = ['종목코드']).fillna('불가')
    신한_최종['대출가능여부'] = '-'
    신한_최종 = 신한_최종[['종목코드', '증거금률', '신용가능여부', '대출가능여부']]
    신한_최종['증거금률'] = 신한_최종['증거금률'] + '%'
    신한_최종 = 신한_최종.rename(columns = {'증거금률' : '신한_증거금률', '신용가능여부' : '신한_신용가능여부', '대출가능여부' : '신한_대출가능여부'})
    
    NH_증거금_최종 = pd.concat(NH_증거금_output).drop_duplicates('종목명')
    NH_신용대출_최종 = pd.concat(NH_신용대출_output).drop_duplicates('종목명')
    NH_최종 = NH_증거금_최종.merge(NH_신용대출_최종, how = 'left', on = '종목명').fillna('불가')
    NH_최종 = NH_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    NH_최종['증거금률'] = NH_최종['증거금률'] + '%'
    NH_최종 = NH_최종.rename(columns = {'증거금률' : 'NH_증거금률', '신용가능여부' : 'NH_신용가능여부', '대출가능여부' : 'NH_대출가능여부'})
    
    메리츠_최종 = pd.concat(메리츠_증거금_output).drop_duplicates('종목명')
    메리츠_최종 = 메리츠_최종[['종목명', '증거금률', '신용가능여부_', '대출가능여부_']]
    메리츠_최종 = 메리츠_최종.rename(columns = {'증거금률' : '메리츠_증거금률', '신용가능여부_' : '메리츠_신용가능여부', '대출가능여부_' : '메리츠_대출가능여부'})
    
    kb_증거금_최종 = pd.concat(kb_증거금_output).drop_duplicates('종목명')
    kb_증거금_최종 = 키움_최종[['종목명']].merge(kb_증거금_최종, how = 'left', on = '종목명').fillna('40')
    kb_신용_최종 = pd.concat(kb_신용_output).drop_duplicates('종목명')
    kb_대출_최종 = pd.concat(kb_대출_output).drop_duplicates('종목명')
    kb_최종 = kb_증거금_최종.merge(kb_신용_최종, how = 'left', on = '종목명')\
                           .merge(kb_대출_최종, how = 'left', on = '종목명').fillna('불가')
    kb_최종 = kb_최종[['종목명', '증거금률', '신용가능여부', '대출가능여부']]
    kb_최종['증거금률'] = kb_최종['증거금률'] + '%'
    kb_최종 = kb_최종.rename(columns = {'증거금률' : 'kb_증거금률', '신용가능여부' : 'kb_신용가능여부', '대출가능여부' : 'kb_대출가능여부'})    

    대신_최종 = pd.concat(대신_증거금_output).drop_duplicates('종목명')
    대신_최종 = 대신_최종[['종목명', '증거금률', '신용가능_', '대출가능_']]
    대신_최종 = 대신_최종.rename(columns = {'증거금률' : '대신_증거금률', '신용가능_' : '대신_신용가능여부', '대출가능_' : '대신_대출가능여부'})
    
    ## 종목 구분 추가
    KOSPI_tickers = stock.get_market_ticker_list(market="KOSPI")
    KOSDAQ_tickers = stock.get_market_ticker_list(market="KOSDAQ")
    ETF_tickers = stock.get_etf_ticker_list()
    
    KOSPI_DF = pd.DataFrame({'ticker' : KOSPI_tickers, '종목구분' : ['KOSPI'] * len(KOSPI_tickers)})
    KOSDAQ_DF = pd.DataFrame({'ticker' : KOSDAQ_tickers, '종목구분' : ['KOSDAQ'] * len(KOSDAQ_tickers)})
    ETF_DF = pd.DataFrame({'ticker' : ETF_tickers, '종목구분' : ['ETF'] * len(ETF_tickers)})
        
    
    stock_df = pd.concat([KOSPI_DF, KOSDAQ_DF, ETF_DF], axis = 0)
    stock_df['종목코드'] = 'A' + stock_df['ticker']
    
    total_output = 키움_최종.merge(stock_df, how = 'left', on = '종목코드')[['종목코드', '종목명', '종목구분', '키움_증거금률', '키움_신용가능여부', '키움_대출가능여부']]\
                           .merge(미래_최종, how = 'left', on = '종목명')\
                           .merge(한투_최종, how = 'left', on = '종목명')\
                           .merge(NH_최종, how = 'left', on = '종목명')\
                           .merge(삼성_최종, how = 'left', on = '종목명')\
                           .merge(신한_최종, how = 'left', on = '종목코드')\
                           .merge(메리츠_최종, how = 'left', on = '종목명')\
                           .merge(하나_최종, how = 'left', on = '종목명')\
                           .merge(kb_최종, how = 'left', on = '종목명')\
                           .merge(대신_최종, how = 'left', on = '종목명')

    ## ETN 제거
    total_output = total_output[~total_output['종목명'].str.contains('ETN')]
    
    total_output['정렬식'] = total_output['키움_증거금률'].str.zfill(4) + total_output['종목코드']
    total_output = total_output.sort_values('정렬식').fillna('조회X')
    total_output2 = total_output.drop('정렬식', axis = 1)
    today = datetime.today()
    total_output2.to_excel(f'total_output_{today.year}.{today.month}.{today.day}.xlsx', index = False)
    print('저장완료!!')

class Securities:
    def __init__(self):
        self.kiwoom = Kiwoom()
        self.kb = KB()
        self.shinhan = Shinhan()
        self.samsung = Samsung()
        self.hankook = Hankook()
        self.hana = Hana()
        self.meritz = Meritz()
        self.mirae = Mirae()
        self.nh = NH()
        self.daishin = Daishin()

def run():
    securities = Securities()
    
    error_s = []
    
    securities_dict = {'nh':'NH', 'samsung':'삼성', 'shinhan':'신한', 'kb':'kb', 'hana':'하나', 'hankook':'한투', 'meritz':'메리츠',
                       'kiwoom':'키움', 'mirae':'미래', 'daishin':'대신'}

    completed = []
    
    os.chdir(r"C:\Users\kiwoom\Documents")
    file_list = os.listdir("./")
    today = datetime.today()
    target_word = f'{today.year}.{today.month}.{today.day}'
    target_file = []
    for file in file_list:
        if target_word in file:
            for v in securities_dict.values():
                if v in file:
                    completed.append(v)

    completed = set(completed)
    print("완료된 증권사 : ", completed)
    
    ## 모든 증권사 실행
    for idx, s in enumerate(securities_dict.keys()):
        print(str(idx + 1) + '.', end = '')
        if securities_dict[s] in completed:
            print(securities_dict[s] + " <-- 파일 저장완료!!")
            print()
            continue
        try:
            getattr(getattr(securities, f'{s}'), 'run')()
        except Exception as err:
            ## error 발생한 증권사 출력 및 저장
            print(s, '<--실행오류 발생')
            print(err)
            error_s.append(s)
            getattr(getattr(securities, f'{s}'), 'close_hts')()
    
            ## 문서파일에서 관련 파일 모두 지우기
            name = securities_dict[s]
            for f in os.listdir("./"):
                if name in f:
                    os.remove('./' + f)
        print()
    
    ## error 난 증권사만 다시 실행
    print('** 에러난 증권사 재실행 **')
    print(error_s)
    for s in error_s:
        try:
            getattr(getattr(securities, f'{s}'), 'run')()
        except:
            ## error 발생한 증권사 출력
            print(s, '<--최종 실행오류 발생')


def check_output():
    os.chdir(r"C:\Users\kiwoom\Documents")
    
    today = datetime.today()
    file_list = os.listdir("./")
    
    target_file_list = []
    for f in file_list:
        if os.path.isdir(f):
            try:
                target_file_list.append(datetime.strptime('20' + f, '%Y.%m.%d'))
            except:
                pass
    
    min_days = datetime(2024, 1, 1) - datetime(1995, 1, 1)
    target_day = None
    for f in target_file_list:
        if today - f < min_days:
            min_days = today - f
            target_day = f
    
    
    target_year = target_day.year
    target_month = target_day.month
    target_day = target_day.day
    
    today_output = pd.read_excel(f'total_output_{today.year}.{today.month}.{today.day}.xlsx')
    recent_output = pd.read_excel(f'{str(target_year)[2:]}.{target_month}.{target_day}/total_output_{target_year}.{target_month}.{target_day}.xlsx')
    
    for c in today_output.columns:
        if c not in ['Unnamed: 0', '종목코드', '종목명', '종목구분', '정렬식']:
            if '증거금률' in c:
                len_recent = len(recent_output[recent_output[c] == '100%'])
                len_today = len(today_output[today_output[c] == '100%'])
    
            else:
                len_recent = len(recent_output[(recent_output[c] != '불가') & (recent_output[c] != '조회X')])
                len_today = len(today_output[(today_output[c] != '불가') & (today_output[c] != '조회X')])
            
            print('최근 : %5s, 오늘 : %5s, 차이(오늘 - 최근) : %5s, %20s' % (len_recent, len_today, len_today - len_recent, (len_today - len_recent) * 100 / len_recent), c)

<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:44: SyntaxWarning: invalid escape sequence '\d'
C:\Users\kiwoom\AppData\Local\Temp\ipykernel_13800\3739159843.py:44: SyntaxWarning: invalid escape sequence '\d'
  m = re.match("(.+?) +(\d+) (.+?) +(\d+) +(\d+.* K).*",task)


In [12]:
run()

완료된 증권사 :  {'kb', '삼성', '하나', '신한', '키움', 'NH', '한투', '미래', '메리츠'}
1.NH <-- 파일 저장완료!!

2.삼성 <-- 파일 저장완료!!

3.신한 <-- 파일 저장완료!!

4.kb <-- 파일 저장완료!!

5.하나 <-- 파일 저장완료!!

6.한투 <-- 파일 저장완료!!

7.메리츠 <-- 파일 저장완료!!

8.키움 <-- 파일 저장완료!!

9.미래 <-- 파일 저장완료!!

10.대신 시작!!
증거금대신_2025.12.24<--Saved!!

** 에러난 증권사 재실행 **
[]


In [13]:
total_output_save()

저장완료!!


In [14]:
check_output()

최근 :  1608, 오늘 :  1608, 차이(오늘 - 최근) :     0,                  0.0 키움_증거금률
최근 :  2071, 오늘 :  2071, 차이(오늘 - 최근) :     0,                  0.0 키움_신용가능여부
최근 :  2080, 오늘 :  2080, 차이(오늘 - 최근) :     0,                  0.0 키움_대출가능여부
최근 :  1621, 오늘 :  1621, 차이(오늘 - 최근) :     0,                  0.0 미래_증거금률
최근 :  1890, 오늘 :  1890, 차이(오늘 - 최근) :     0,                  0.0 미래_신용가능여부
최근 :  3842, 오늘 :  3842, 차이(오늘 - 최근) :     0,                  0.0 미래_대출가능여부
최근 :  2627, 오늘 :  2627, 차이(오늘 - 최근) :     0,                  0.0 한투_증거금률
최근 :  1219, 오늘 :  1219, 차이(오늘 - 최근) :     0,                  0.0 한투_신용가능여부
최근 :  3845, 오늘 :  3845, 차이(오늘 - 최근) :     0,                  0.0 한투_대출가능여부
최근 :  2044, 오늘 :  2044, 차이(오늘 - 최근) :     0,                  0.0 NH_증거금률
최근 :  1720, 오늘 :  1720, 차이(오늘 - 최근) :     0,                  0.0 NH_신용가능여부
최근 :  1744, 오늘 :  1744, 차이(오늘 - 최근) :     0,                  0.0 NH_대출가능여부
최근 :   944, 오늘 :   944, 차이(오늘 - 최근) :     0,                  0.0 삼성_증거금률
최근 :  1717, 오늘 :  1717

C:\Users\kiwoom\AppData\Local\Temp\ipykernel_13800\3739159843.py:44: SyntaxWarning: invalid escape sequence '\d'
  m = re.match("(.+?) +(\d+) (.+?) +(\d+) +(\d+.* K).*",task)


ZeroDivisionError: division by zero

In [ ]:

# os.chdir(r'\\Kiwoom-TF\종목 증거금률 및 신용가능여부\결과 파일')
# os.mkdir('test')증거금메리츠_2025.11.25코스피


In [ ]:
# ## 키움 실행
# kiwoom = Kiwoom()
# kiwoom.run()

In [ ]:
## run()

## 